In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow==2.8.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 38.6 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: goo

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# Set the dataset directory
dataset_dir = '/content/drive/MyDrive/Food Classification'

# Data generators with augmentations
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% validation split
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Same validation split as training
)

# Load the training data
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training',  # Set as training data
    shuffle=True
)

# Load the validation data
validation_generator = validation_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Set as validation data
)

# Build a basic CNN model
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))  # Set dropout rate to 0.4 for regularization

# Output layer with number of classes
model.add(Dense(train_generator.num_classes, activation='softmax'))

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),  # Initial learning rate for faster convergence
    metrics=['accuracy']
)

# Callbacks for saving the best model and reducing the learning rate
checkpoint = ModelCheckpoint('/content/drive/MyDrive/best_cnn_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=21,  # Based on 656 training images with batch size of 32
    epochs=30,  # Train for 30 epochs
    validation_data=validation_generator,
    validation_steps=6,  # Based on 164 validation images with batch size of 32
    callbacks=[checkpoint, reduce_lr]
)

# Save the trained model
model_path = '/content/drive/MyDrive/Food_Classification_CNN.h5'
model.save(model_path)

print(f"Model saved at {model_path}")


Found 656 images belonging to 8 classes.
Found 164 images belonging to 8 classes.
Epoch 1/30
21/21 [==============================] - 492s 23s/step - loss: 2.1100 - accuracy: 0.1387 - val_loss: 1.9467 - val_accuracy: 0.1951 - lr: 0.0010
Epoch 2/30
21/21 [==============================] - 52s 2s/step - loss: 1.8990 - accuracy: 0.2180 - val_loss: 1.7666 - val_accuracy: 0.2683 - lr: 0.0010
Epoch 3/30
21/21 [==============================] - 50s 2s/step - loss: 1.7921 - accuracy: 0.2607 - val_loss: 1.7921 - val_accuracy: 0.2683 - lr: 0.0010
Epoch 4/30
21/21 [==============================] - 51s 2s/step - loss: 1.7600 - accuracy: 0.2790 - val_loss: 1.6426 - val_accuracy: 0.3720 - lr: 0.0010
Epoch 5/30
21/21 [==============================] - 52s 2s/step - loss: 1.7022 - accuracy: 0.3232 - val_loss: 1.5665 - val_accuracy: 0.4390 - lr: 0.0010
Epoch 6/30
21/21 [==============================] - 53s 3s/step - loss: 1.6710 - accuracy: 0.3262 - val_loss: 1.4730 - val_accuracy: 0.4817 - lr: 0.001